# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!


In [1]:
# imports
import ollama
import os
from IPython.display import Markdown,display
import time

# librerias para claude
import anthropic
from dotenv import load_dotenv

# libreria de Gradio
import gradio as gr


In [2]:
# constantes

MODEL_Claude = 'claude-3-haiku-20240307'

MODEL_LLAMA = 'llama3.2'

In [3]:
# obteniendo la variable de api de antropic
load_dotenv()

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if anthropic_api_key:
    print('se cargo correctamente: {}'.format(anthropic_api_key[:4]))
else:
    print("No se encontro.")

se cargo correctamente: sk-a


In [4]:
# genero el objeto de claude
claude = anthropic.Anthropic()

In [24]:
# Configuracion de claude responda, con streaming

pront_system = "Eres un asistente que ayudas a entender codigo de python"
pront_user = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""




def stream_claude(pront):
    
    resultado = claude.messages.stream(model=MODEL_Claude,
                                       messages= [{'role':'user','content':pront}],
                                      system=pront_system,
                                      max_tokens=600,
                                      temperature= 0.7)
    response = ""
    with resultado as stream:
        for text in  stream.text_stream:
            response += text or ''
            yield response         
    
view = gr.Interface(fn=stream_claude,
                    inputs = [gr.Textbox(label='Tu mensaje:')],
                    outputs = [gr.Markdown(label="Respuesta:")],
                    flagging_mode = 'never'
                   )

In [26]:
# Haz que claude responda
view.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [30]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

question = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""


In [28]:
# Configura el entorno para ollama
def tutor(prompt):
    mensaje = [{
        'role': 'user',
        'content': f"""me ayudaras como tutor en mi proceso de formación de programación en python para ML. {prompt}
        Explicamelo de una forma clara y corta."""
    }]
    
    response_stream = ollama.chat(
        model=MODEL_LLAMA,
        messages=mensaje,
        stream=True
    )
    
    texto_completo = ""
    for chunk in (chunk['message']['content'] for chunk in response_stream if 'message' in chunk):
        texto_completo += chunk
        display(Markdown(texto_completo), clear=True)
        time.sleep(0.02)
    


In [31]:
# Haz que Llama 3.2 responda

tutor(question)

¡Claro que sí! Me alegra poder ayudarte en tu proceso de formación.

El código que tienes es un ejemplo común en Python, relacionado con la iteración y generadores. Vamos a desglosarlo paso a paso:

**¿Qué hace este código?**

Este código utiliza una técnica llamada "generador" (también conocida como "yield from") para generar un conjunto de valores a partir de otra colección.

**Explicación del código:**

* `yield from`: Es una directiva que permite al programador crear un generator (un objeto que puede ser iterado) desde otro objecto.
* `{book.get("author") for book in books if book.get("author")}`: Esto es una expresión genérica que utiliza el operador `for` para iterar sobre los libros (`books`) en la colección, y para cada libro, obtiene el autor utilizando la función `get("author")`.

**Cómo funciona el código:**

1. El generator se crea cuando se ejecuta la directiva `yield from`.
2. La expresión genérica `{book.get("author") for book in books if book.get("author")}` es evaluada y se convierte en un generator.
3. Cuando se le pide a este generator que "retorne" los valores, utiliza el operador `for` para iterar sobre cada libro (`books`) y obtener su autor.
4. El resultado final es un conjunto de valores (los autores) generados a partir del conjunto de libros.

**Por qué funciona**

El uso de la directiva `yield from` permite al generator acceder a los valores del otro objecto (`books`) sin tener que recogerlos en una lista o variable intermedia. Esto hace que el código sea más eficiente y memoria eficiente, especialmente cuando se trabaja con grandes conjuntos de datos.

En resumen, este código utiliza un generador para obtener los autores de cada libro en la colección `books`, sin tener que recogerlos en una lista o variable intermedia.